In [34]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle as pkl
from pymongo import MongoClient

MONGO_PORT = 27017
MONGO_DB = 'entrdir'
MONGO_COLLECTION = 'employee'



In [35]:
client = MongoClient('localhost', MONGO_PORT)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]
employees = list(collection.find())
employees = pd.DataFrame(employees)
target_features_fields = employees[['salary','role','work_location']]
target_features_fields = target_features_fields.dropna()
target_features_fields = target_features_fields.drop_duplicates()


In [36]:
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(target_features_fields[['role','work_location']])
X_encoded = pd.DataFrame(X_encoded)
y= target_features_fields['salary']
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)




In [37]:
with open('model_dump.pkl', 'wb') as f:
    pkl.dump(model, f)

with open('encoder_dump.pkl', 'wb') as f:
    pkl.dump(encoder, f)